# IMPORT

In [ ]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk

from tkinter import ttk
from tkinter import *

# Load Datasets

In [ ]:
data_path = 'datasets/Datafull train.csv'

df = pd.read_csv(data_path, delimiter=";")

df.head()

In [ ]:
# cek dimensi data sebelum masuk data preprocessing
df.shape

In [ ]:
df.nunique() # mengecek data unique pada setiap kolom

In [ ]:
df.info()

# Data PreProcessing

In [ ]:
df.isna().sum() # mengecek data yang kosong
df.duplicated().sum # mengecek data yang duplikat

df = df.drop_duplicates() #menghapus data duplikat
df = df.fillna(df.mode().iloc[0])

In [42]:
df.shape

(32849, 43)

In [43]:
df.nunique

<bound method DataFrame.nunique of             0x       0y       1x       1y       2x       2y       3x       3y  \
0      449.894  638.530  329.537  588.029  262.366  475.709  253.967  343.421   
1      459.426  638.641  347.614  599.550  273.522  486.969  262.374  354.810   
2      444.194  659.268  326.598  616.910  253.226  497.914  243.082  361.876   
3      443.780  650.429  324.970  607.825  252.818  487.201  244.345  356.561   
4      452.860  638.769  340.436  595.407  272.323  487.497  254.365  357.569   
...        ...      ...      ...      ...      ...      ...      ...      ...   
32844  438.216  475.728  384.271  438.175  365.128  370.238  359.100  308.232   
32845  451.700  471.125  388.505  420.364  370.429  366.011  359.194  301.358   
32846  461.909  469.288  400.524  424.948  372.224  372.311  353.600  310.913   
32847  451.633  470.803  393.561  427.040  365.993  371.472  349.987  306.755   
32848  446.508  479.837  383.399  422.028  364.069  359.685  355.280  293.